# Imports

In [1]:
from pyiron_contrib.tinybase.node import AbstractNode, FunctionNode, SeriesNode, LoopNode

/home/ponder/science/phd/dev/pyiron_contrib/pyiron_contrib/__init__.py:9: UserWarning: pyiron module not found, importing Project from pyiron_base
  warnings.warn("pyiron module not found, importing Project from pyiron_base")


In [2]:
from pyiron_contrib.tinybase.executor import ProcessExecutor, BackgroundExecutor, Executor

In [3]:
import logging
logging.getLogger().setLevel(20)

In [4]:
import numpy as np

# Function Node

## Basic

In [5]:
def calc_fib(n):
    import time
    n1 = n2 = 1
    for i in range(n):
        time.sleep(.1)
        x = n1 + n2
        n1 = n2
        n2 = x
    return x

In [6]:
f = FunctionNode(calc_fib)

In [7]:
f.input.storage

DataContainer([])

In [8]:
f.input.args

[]

In [9]:
f.input.kwargs

{}

In [10]:
f.input.kwargs['n'] = 10

In [11]:
f.input.kwargs

{'n': 10}

In [12]:
exe = f.run()

In [13]:
exe._run_machine._data

{'status': (ReturnStatus(Code.DONE, None),),
 'output': (<pyiron_contrib.tinybase.node.FunctionOutput at 0x7f12bb0a3d50>,)}

In [14]:
exe.output[0].result

144

In [15]:
exe._run_time, exe._collect_time

(1.0013468409888446, 2.5904009817168117e-05)

## Do the same but in the background

In [16]:
f = FunctionNode(calc_fib)

In [17]:
f.input.kwargs['n'] = 100

In [18]:
exe = f.run(how='background')

In [19]:
exe._run_machine._data

{}

In [20]:
exe._run_machine.state

<Code.RUNNING: 'running'>

In [21]:
exe.wait()

In [22]:
exe.output[0].result

927372692193078999176

## Do the same but in the background as process

In [23]:
fib_node = FunctionNode(calc_fib)

In [24]:
fib_node.input.kwargs['n'] = 100

In [25]:
exe = fib_node.run(how='process')

In [26]:
exe._run_machine._data

{}

In [27]:
exe._run_machine.state

<Code.RUNNING: 'running'>

In [28]:
exe.wait()

In [29]:
exe.output[0].result

927372692193078999176

# Executors handle single nodes and lists of them on the same footing

In [69]:
nodes = [FunctionNode(calc_fib) for _ in range(10)]

In [72]:
for i, n in enumerate(nodes):
    n.input.kwargs['n'] = 3 + i

## With the basic executor

In [73]:
exe = Executor(nodes)
exe.run()

In [74]:
exe.output

(<pyiron_contrib.tinybase.node.FunctionOutput at 0x7f12bae7f050>,
 <pyiron_contrib.tinybase.node.FunctionOutput at 0x7f12b9fb6e50>)

In [75]:
exe.output[5].result

55

## With the process executor

In [88]:
exe = ProcessExecutor(nodes)
exe.run()

In [89]:
exe.wait()

In [90]:
exe.status

[ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None)]

In [91]:
exe.output

In [92]:
exe.output[5].result

55

# SeriesNode

In [30]:
s = SeriesNode()

In [31]:
f1 = FunctionNode(calc_fib)

In [32]:
f2 = FunctionNode(np.sqrt)

In [62]:
def transfer(input, output):
    input.args = [output.result]

In [63]:
s.input.first(f1).then(f2, transfer)

In [64]:
s.input.nodes[0].input.kwargs['n'] = 10

In [65]:
status, output = s.execute()

In [66]:
status

ReturnStatus(Code.DONE, None)

In [67]:
output.result

12.0

# Loop Node

## Simple repeat loop

In [38]:
l = LoopNode()

In [39]:
l.input.node = FunctionNode(lambda: np.random.rand())

In [40]:
l.input.repeat(10, restart=lambda output, input, scratch: print(output.result))

In [41]:
exe = l.run()

0.06831008568469543
0.4397644993497386
0.4710237017187203
0.6351871327200411
0.09403372149094191
0.45571674954951835
0.8693040125694965
0.03592129945541278
0.2842080026440601


In [42]:
exe.status

(ReturnStatus(Code.DONE, None),)

## Loop with a termination condition

In [93]:
l = LoopNode()

In [94]:
l.input.node = FunctionNode(lambda: np.random.rand())

In [99]:
l.input.control_with(
    condition=lambda node, output, scratch: output.result < .05,
    restart=lambda output, input, scratch: print(output.result)
)

In [100]:
exe = l.run()

0.6180549529309188
0.9433407283326789
0.6650129766719227
0.9246096624581522
0.702924921444492
0.4932765584360923
0.10209510690867707
0.19723819666451714
0.6319420933414326
0.9376161926340415
0.1911762555082791
0.7812203005244642
0.36658807729956766
0.997651587491596
0.17214861003243775
0.49700246072622345
0.8929166329882523
0.9069634041837235
0.928329630027329
0.14530372536131697
0.4551759858923593
0.8299354186855429
0.9971370925238271
0.3922295916439884
0.43629137886178726
0.13481396015844416
0.06396401175605293
0.0502648932556814
0.0919464823724655
0.2478196375875663
0.5547919839305524
0.9950273201349219
0.7490433592510488
0.5708404460188841
0.2800227217981094
0.452859610657651
0.5086825431878267
0.7486390124589416
0.34312007912192777
0.771168396478236
0.4539288607160801
0.7642828950901653
0.9944398067831015
0.8876987515750713
0.7498600155938839
0.8124747754930199
0.9020421405237081
0.40694715335295206
0.6880129743298647
0.8457057679143185
0.10612064010204925
0.4658543363818123
0.359

In [101]:
exe.status[0]

ReturnStatus(Code.DONE, None)

In [102]:
exe.output[0].result

0.022068342941342967